In [1]:
import os
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\kidney_disease\\research'

In [2]:
os.chdir('../')
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\kidney_disease'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size:list
    params_learning_rate:float
    params_include_top:bool
    params_weights:float
    params_classes:int
    

In [4]:
from kidney_disease.constant import *
from kidney_disease.utils.common import create_directories,read_yaml

In [ ]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def prepare_base_model_config(self)->PrepareBaseModelConfig:
        config=self.config.prepare_base_model

        create_directories([config.root_dir])

        base_model=PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
        )
        return base_model

In [6]:
import zipfile
import urllib.request as request
import tensorflow as tf

In [7]:
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config=config

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top,
            weights=self.config.params_weights
        )
        self.save_model(path=self.config.base_model_path,model=self.model)
    
    def _prepare_base_model(self,free_all,freeze_till,model,learning_rate,classes):
        if free_all:
            for layer in model.layers:
                model.trainable=False
        elif (freeze_till is not None) and freeze_till>0:
            for layer in model.layers[:-freeze_till]:
                layer.trainable=False
        
        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(
            activation="softmax",
            units=classes
        )(flatten_in)

        full_model=tf.keras.models.Model(inputs=model.input,outputs=prediction)

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model=self._prepare_base_model(
            free_all=True,
            freeze_till=None,
            model=self.model,
            learning_rate=self.config.params_learning_rate,
            classes=self.config.params_classes
        )
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)

In [9]:

try:
    config=ConfigurationManager()
    prepare_base_model_config=config.prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except ValueError as e:
    raise e

Loading configuration from config\config.yaml and params.yaml
[2025-04-10 22:56:38,794: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-10 22:56:38,799: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-10 22:56:38,803: INFO: common: created directory at: artifacts]
[2025-04-10 22:56:38,809: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 9s 0us/step
[2025-04-10 22:56:49,544: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)    